In [ ]:
            # Find "Next Page"
            next_page_link = soup.find('div', class_=NEXT_PAGE_CLASS)
            if next_page_link and next_page_link.find('a'):
                current_url = next_page_link.find('a')['href']
            else:
                print("\nAll done! Scrape complete.")
                break

In [ ]:
# 1. Upgrade libraries (Standard fix for Colab warnings)
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

from googleapiclient.discovery import build
from google.auth import default
from google.colab import auth
from datetime import datetime
import re

# 2. Authenticate
try:
    auth.authenticate_user()
    creds, _ = default()
    drive_service = build('drive', 'v3', credentials=creds)
    print("✅ Authentication successful.")
except Exception as e:
    print(f"❌ Authentication failed: {e}")

# 3. CONFIGURATION
# Replace with your own Google Drive folder IDs
# SOURCE_FOLDER_ID: The folder containing the PDF files to convert
SOURCE_FOLDER_ID = 'YOUR_SOURCE_FOLDER_ID_HERE'
# DEST_FOLDER_ID: The folder where the converted Google Docs will be saved
DEST_FOLDER_ID   = 'YOUR_DEST_FOLDER_ID_HERE'

def get_sermon_date(file_item):
    """
    Robustly parses dates. Handles errors without crashing.
    """
    filename = file_item['name']

    # Regex for MM-DD-YY or MM.DD.YY
    match = re.search(r"(\d{1,2})[-.](\d{1,2})[-.](\d{2,4})", filename)

    if match:
        p1, p2, year_str = map(str, match.groups())
        year = int(year_str)
        if year < 100: year += 2000

        # Try formatting as Month-Day (Standard US)
        try:
            return datetime(year, int(p1), int(p2))
        except ValueError:
            # If that fails (e.g. Month is 13), try Day-Month (European/Other)
            try:
                return datetime(year, int(p2), int(p1))
            except ValueError:
                print(f"   ⚠️ Warning: Could not read date from '{filename}'. Ignoring date.")
                pass

    return datetime(1900, 1, 1)

def convert_latest_10():
    print(f"\n👀 Verifying access to Source Folder...")
    try:
        folder = drive_service.files().get(fileId=SOURCE_FOLDER_ID).execute()
        print(f"   ✅ Success! Found folder: '{folder.get('name')}'")
    except Exception as e:
        print(f"   ❌ ERROR: Cannot access source folder. {e}")
        return

    print("🔍 Listing PDFs...")
    query = f"'{SOURCE_FOLDER_ID}' in parents and trashed = false"

    try:
        results = drive_service.files().list(
            q=query, pageSize=1000, fields="files(id, name, mimeType)"
        ).execute()
        files = results.get('files', [])
    except Exception as e:
        print(f"   ❌ Query Error: {e}")
        return

    # Filter PDFs
    pdf_files = [f for f in files if 'pdf' in f['mimeType'].lower()]

    if not pdf_files:
        print("   ⚠️ No PDFs found.")
        return

    # Sort files by Date (Newest First)
    files_sorted = sorted(pdf_files, key=get_sermon_date, reverse=True)
    latest_10 = files_sorted[:10]

    print(f"📅 Found {len(pdf_files)} PDFs. Converting the latest {len(latest_10)}:\n")

    for i, file in enumerate(latest_10, 1):
        original_name = file['name']
        new_name = original_name.replace('.pdf', '').replace('.PDF', '')

        sermon_date = get_sermon_date(file)
        # Handle cases where date parsing failed completely
        if sermon_date.year == 1900:
            date_str = "Unknown Date"
        else:
            date_str = sermon_date.strftime('%Y-%m-%d')

        print(f"{i}. [{date_str}] Converting: {original_name}...")

        try:
            body = {
                'name': new_name,
                'parents': [DEST_FOLDER_ID],
                'mimeType': 'application/vnd.google-apps.document'
            }
            drive_service.files().copy(
                fileId=file['id'],
                body=body
            ).execute()
            print("   ✅ Done.")
        except Exception as e:
            print(f"   ❌ Error: {e}")

    print("\n🎉 All finished!")

convert_latest_10()